In [1]:
import json
from typing import TypedDict

import numpy as np
from constants import DATA_DIR
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.early_stop import no_progress_loss
from hyperopt.pyll.base import scope
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

from astrofit.utils import AsteroidLoader

In [2]:
asteroid_loader = AsteroidLoader(DATA_DIR)

ASTEROIDS_FREQ_DATA_PATH = DATA_DIR / "asteroids_freq_data.json"

In [3]:
with open(ASTEROIDS_FREQ_DATA_PATH, "r") as f:
    asteroids_freq_data = json.load(f)

In [4]:
config, asteroids_data = asteroids_freq_data["config"], asteroids_freq_data["asteroids"]

In [5]:
filtered_data = {name: data for name, data in asteroids_data.items() if not data["is_failed"]}
print(f"Filtered {len(filtered_data)} asteroids ({100*(len(asteroids_data) - len(filtered_data)) / len(asteroids_data):.2f}% failed)")

Filtered 2662 asteroids (40.41% failed)


In [6]:
class AsteroidData(TypedDict):
    is_failed: bool
    reason: str | None
    period: float
    processing_time: float
    freq_features: list[list]  # 1 - 4 sequences of 50 floats (freqs) from 0 to 12
    pow_features: list[list]  # 1 - 4 sequences of 50 floats (pows) from 0 to 1 (the same shape as freq_features)

In [7]:
filtered_data = {name: AsteroidData(**data) for name, data in filtered_data.items()}

In [8]:
np.percentile([data["period"] for data in filtered_data.values()], [0, 5, 25, 50, 75, 95, 100])

array([ 2.134122 ,  3.5124675,  5.3783405,  8.004365 , 12.93966  ,
       25.9748   , 39.8999   ])

In [9]:
train_keys, val_test_keys = train_test_split(list(filtered_data.keys()), test_size=0.2, random_state=884288)
val_keys, test_keys = train_test_split(val_test_keys, test_size=0.33, random_state=884288)

print(f"Train: {len(train_keys)} asteroids ({100 * len(train_keys) / len(filtered_data):.2f})")
print(f"Validation: {len(val_keys)} asteroids ({100*len(val_keys) / len(filtered_data):.2f})")
print(f"Test: {len(test_keys)} asteroids ({100 * len(test_keys) / len(filtered_data):.2f})")

Train: 2129 asteroids (79.98)
Validation: 357 asteroids (13.41)
Test: 176 asteroids (6.61)


In [10]:
train_set, val_set, test_set = (
    {key: filtered_data[key] for key in train_keys},
    {key: filtered_data[key] for key in val_keys},
    {key: filtered_data[key] for key in test_keys},
)

In [11]:
print("Train periods:")
print(np.percentile([data["period"] for data in train_set.values()], [0, 5, 25, 50, 75, 95, 100]))

print("\nValidation periods:")
print(np.percentile([data["period"] for data in val_set.values()], [0, 5, 25, 50, 75, 95, 100]))

print("\nTest periods:")
print(np.percentile([data["period"] for data in test_set.values()], [0, 5, 25, 50, 75, 95, 100]))

Train periods:
[ 2.134122  3.54028   5.385282  8.04945  13.0339   26.0133   39.8999  ]

Validation periods:
[ 2.68212    3.4877328  5.363025   7.81773   12.68499   25.74368
 38.7804   ]

Test periods:
[ 2.738963    3.70729275  5.432665    7.776705   12.583375   25.9855
 39.848     ]


In [12]:
def _extract_features(data_set: dict[str, AsteroidData]) -> tuple[list[np.ndarray], list[np.ndarray]]:
    freq_features = [np.array(sample["freq_features"]) for sample in data_set.values()]
    pow_features = [np.array(sample["pow_features"]) for sample in data_set.values()]
    return freq_features, pow_features


def _standardize_features(features: list[np.ndarray], scaler: StandardScaler) -> list[np.ndarray]:
    flattend = np.concatenate(features)
    transformed = scaler.transform(flattend)

    result = []
    start = 0
    for seq in features:
        end = start + len(seq)
        result.append(transformed[start:end])  # type: ignore
        start = end
    
    return result


# Combine freq and pow features
def _combine_features(freq: list[np.ndarray], pow: list[np.ndarray]) -> list[np.ndarray]:
    return [np.stack((f, p), axis=-1) for f, p in zip(freq, pow)]


def standardize_asteroid_data(
    train_set: dict[str, AsteroidData], val_set: dict[str, AsteroidData], test_set: dict[str, AsteroidData]
) -> tuple[list[np.ndarray], list[np.ndarray], list[np.ndarray], StandardScaler, StandardScaler]:
    train_freq, train_pow = _extract_features(train_set)
    val_freq, val_pow = _extract_features(val_set)
    test_freq, test_pow = _extract_features(test_set)

    # Flatten the features for fitting the scalers
    train_freq_flat = np.concatenate(train_freq)
    train_pow_flat = np.concatenate(train_pow)

    # Create and fit scalers
    freq_scaler = StandardScaler()
    pow_scaler = StandardScaler()

    freq_scaler.fit(train_freq_flat)
    pow_scaler.fit(train_pow_flat)

    # Standardize all sets
    train_freq_std = _standardize_features(train_freq, freq_scaler)
    train_pow_std = _standardize_features(train_pow, pow_scaler)
    val_freq_std = _standardize_features(val_freq, freq_scaler)
    val_pow_std = _standardize_features(val_pow, pow_scaler)
    test_freq_std = _standardize_features(test_freq, freq_scaler)
    test_pow_std = _standardize_features(test_pow, pow_scaler)

    train_std = _combine_features(train_freq_std, train_pow_std)
    val_std = _combine_features(val_freq_std, val_pow_std)
    test_std = _combine_features(test_freq_std, test_pow_std)

    return train_std, val_std, test_std, freq_scaler, pow_scaler


def prepare_for_xgboost(data: list[np.ndarray], max_len: int = 4) -> np.ndarray:
    # padded_data = np.full((len(data), max_len * 100), np.nan)

    # OLD IMPLEMENTATION
    
    concatenated = [sample.reshape(-1, 100) for sample in data]
    padded = [np.pad(seq, ((0, max_len - len(seq)), (0, 0)), mode="constant", constant_values=np.nan) for seq in concatenated]

    return np.stack(padded).reshape(-1, 400)

In [13]:
train_std, val_std, test_std, freq_scaler, pow_scaler = standardize_asteroid_data(train_set, val_set, test_set)

X_train = prepare_for_xgboost(train_std)
X_val = prepare_for_xgboost(val_std)
X_test = prepare_for_xgboost(test_std)

y_train = np.array([asteroid["period"] for asteroid in train_set.values()])
y_val = np.array([asteroid["period"] for asteroid in val_set.values()])
y_test = np.array([asteroid["period"] for asteroid in test_set.values()])

In [14]:
model = XGBRegressor(
    early_stopping_rounds=100,
    n_estimators=100,
    objective="reg:squarederror",
    missing=np.nan,
)
model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True,
)

# # Make predictions
predictions = model.predict(X_test)

[0]	validation_0-rmse:7.07857
[1]	validation_0-rmse:6.98654
[2]	validation_0-rmse:6.96357
[3]	validation_0-rmse:7.05208
[4]	validation_0-rmse:6.99748
[5]	validation_0-rmse:7.00773
[6]	validation_0-rmse:7.00307
[7]	validation_0-rmse:6.98369
[8]	validation_0-rmse:6.96335
[9]	validation_0-rmse:6.96571
[10]	validation_0-rmse:6.97979
[11]	validation_0-rmse:6.98959
[12]	validation_0-rmse:7.02614
[13]	validation_0-rmse:7.01322
[14]	validation_0-rmse:7.00392
[15]	validation_0-rmse:7.04181
[16]	validation_0-rmse:7.03805
[17]	validation_0-rmse:7.02167
[18]	validation_0-rmse:7.00932
[19]	validation_0-rmse:7.02791
[20]	validation_0-rmse:7.01104
[21]	validation_0-rmse:7.01789
[22]	validation_0-rmse:7.01972
[23]	validation_0-rmse:7.01291
[24]	validation_0-rmse:6.99187
[25]	validation_0-rmse:6.98453
[26]	validation_0-rmse:6.99433
[27]	validation_0-rmse:7.00461
[28]	validation_0-rmse:7.00834
[29]	validation_0-rmse:7.04024
[30]	validation_0-rmse:7.03474
[31]	validation_0-rmse:7.01991
[32]	validation_0-

In [15]:
space = {
    "max_depth": scope.int(hp.quniform("max_depth", 3, 18, 1)),
    "gamma": hp.uniform("gamma", 1, 9),
    "reg_alpha": hp.quniform("reg_alpha", 1, 100, 1),
    "reg_lambda": hp.quniform("reg_lambda", 1, 100, 1),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "min_child_weight": hp.quniform("min_child_weight", 1, 10, 1),
    "n_estimators": scope.int(hp.quniform("n_estimators", 100, 2000, 1)),
    "eta": hp.uniform("eta", 0.01, 0.3),
    "tree_method": hp.choice("tree_method", ["auto", "exact", "approx", "hist"]),
    "max_leaves": scope.int(hp.uniform("max_leaves", 2, 2**10)),
    "max_bin": scope.int(hp.uniform("max_bin", 2, 2**10)),
    "num_parallel_tree": scope.int(hp.quniform("num_parallel_tree", 1, 10, 1)),
    "early_stopping_rounds": scope.int(hp.uniform("early_stopping_rounds", 10, 100)),
}

In [16]:
train_std, val_std, test_std, freq_scaler, pow_scaler = standardize_asteroid_data(train_set, val_set, test_set)

X_train = prepare_for_xgboost(train_std)
X_val = prepare_for_xgboost(val_std)
X_test = prepare_for_xgboost(test_std)

y_train = np.array([asteroid["period"] for asteroid in train_set.values()])
y_val = np.array([asteroid["period"] for asteroid in val_set.values()])
y_test = np.array([asteroid["period"] for asteroid in test_set.values()])

In [17]:
import logging

logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

logger = logging.getLogger(__name__)

In [18]:
def objective(space):
    model = XGBRegressor(
        **space,
        n_jobs=-1,
        random_state=8888,
    )

    logger.debug(f"Training with hyperparameters: {space}")

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        verbose=False,
    )

    logger.debug("Predicting on validation set")

    predictions = model.predict(X_val)
    mse = mean_squared_error(y_val, predictions)

    for pred, true in zip(predictions[:10], y_val[:10]):
        logger.debug(f"Predicted: {pred}, True: {true}")

    logger.debug(f"Validation MSE: {mse}")

    return {"loss": mse, "status": STATUS_OK}


trials = Trials()

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    # early_stop_fn=no_progress_loss(iteration_stop_count=50, percent_increase=0),
)

print("Best hyperparameters:", best)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2024-07-06 18:49:36,453 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.002295 seconds
2024-07-06 18:49:36,454 - hyperopt.tpe - INFO - TPE using 0 trials
2024-07-06 18:49:36,457 - __main__ - DEBUG - Training with hyperparameters: {'colsample_bytree': 0.9969476235882737, 'early_stopping_rounds': 41, 'eta': 0.0120250101497338, 'gamma': 4.061833904618148, 'max_bin': 754, 'max_depth': 14, 'max_leaves': 414, 'min_child_weight': 6.0, 'n_estimators': 351, 'num_parallel_tree': 8, 'reg_alpha': 57.0, 'reg_lambda': 13.0, 'tree_method': 'auto'}
2024-07-06 18:58:20,661 - __main__ - DEBUG - Predicting on validation set
2024-07-06 18:58:20,678 - __main__ - DEBUG - Predicted: 10.95658016204834, True: 34.029
2024-07-06 18:58:20,678 - __main__ - DEBUG - Predicted: 12.948179244995117, True: 7.92812
2024-07-06 18:58:20,679 - __main__ - DEBUG - Predicted: 8.761702537536621, True: 21.3639
2024-07-06 18:58:20,679 - __main__ - DEBUG - Predicted: 8.950424194335938, True: 5.06904
2024-07-06 18:58:20,680 

  2%|▏         | 1/50 [08:44<7:08:07, 524.23s/trial, best loss: 47.61745548559845]

2024-07-06 18:58:20,687 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.001878 seconds
2024-07-06 18:58:20,688 - hyperopt.tpe - INFO - TPE using 1/1 trials with best loss 47.617455
2024-07-06 18:58:20,690 - __main__ - DEBUG - Training with hyperparameters: {'colsample_bytree': 0.6360137106428702, 'early_stopping_rounds': 16, 'eta': 0.01840765817836673, 'gamma': 8.17609301515002, 'max_bin': 964, 'max_depth': 15, 'max_leaves': 249, 'min_child_weight': 10.0, 'n_estimators': 1916, 'num_parallel_tree': 1, 'reg_alpha': 65.0, 'reg_lambda': 76.0, 'tree_method': 'hist'}
2024-07-06 18:58:40,735 - __main__ - DEBUG - Predicting on validation set
2024-07-06 18:58:40,740 - __main__ - DEBUG - Predicted: 12.145458221435547, True: 34.029
2024-07-06 18:58:40,741 - __main__ - DEBUG - Predicted: 12.869976997375488, True: 7.92812
2024-07-06 18:58:40,741 - __main__ - DEBUG - Predicted: 8.34727668762207, True: 21.3639
2024-07-06 18:58:40,742 - __main__ - DEBUG - Predicted: 10.15438461303711, True: 5.0

  4%|▍         | 2/50 [09:04<3:02:07, 227.66s/trial, best loss: 47.13529947301507]

...

 96%|█████████▌| 48/50 [59:08<02:12, 66.42s/trial, best loss: 44.98504367675202]

2024-07-06 19:48:45,160 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.001979 seconds
2024-07-06 19:48:45,160 - hyperopt.tpe - INFO - TPE using 48/48 trials with best loss 44.985044
2024-07-06 19:48:45,177 - __main__ - DEBUG - Training with hyperparameters: {'colsample_bytree': 0.8780660738144379, 'early_stopping_rounds': 22, 'eta': 0.1999653625084797, 'gamma': 6.29507718635026, 'max_bin': 466, 'max_depth': 8, 'max_leaves': 416, 'min_child_weight': 1.0, 'n_estimators': 1995, 'num_parallel_tree': 6, 'reg_alpha': 99.0, 'reg_lambda': 51.0, 'tree_method': 'hist'}
2024-07-06 19:49:04,028 - __main__ - DEBUG - Predicting on validation set
2024-07-06 19:49:04,030 - __main__ - DEBUG - Predicted: 13.53007698059082, True: 34.029
2024-07-06 19:49:04,031 - __main__ - DEBUG - Predicted: 14.266618728637695, True: 7.92812
2024-07-06 19:49:04,031 - __main__ - DEBUG - Predicted: 8.81100082397461, True: 21.3639
2024-07-06 19:49:04,031 - __main__ - DEBUG - Predicted: 8.491029739379883, True: 5.069

 98%|█████████▊| 49/50 [59:27<00:52, 52.16s/trial, best loss: 44.98504367675202]

2024-07-06 19:49:04,036 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.001913 seconds
2024-07-06 19:49:04,037 - hyperopt.tpe - INFO - TPE using 49/49 trials with best loss 44.985044
2024-07-06 19:49:04,051 - __main__ - DEBUG - Training with hyperparameters: {'colsample_bytree': 0.8428712587800602, 'early_stopping_rounds': 31, 'eta': 0.13415665758493056, 'gamma': 7.010578789166178, 'max_bin': 382, 'max_depth': 12, 'max_leaves': 520, 'min_child_weight': 4.0, 'n_estimators': 359, 'num_parallel_tree': 7, 'reg_alpha': 34.0, 'reg_lambda': 75.0, 'tree_method': 'hist'}
2024-07-06 19:50:04,901 - __main__ - DEBUG - Predicting on validation set
2024-07-06 19:50:04,904 - __main__ - DEBUG - Predicted: 12.78778076171875, True: 34.029
2024-07-06 19:50:04,904 - __main__ - DEBUG - Predicted: 13.750221252441406, True: 7.92812
2024-07-06 19:50:04,905 - __main__ - DEBUG - Predicted: 8.878832817077637, True: 21.3639
2024-07-06 19:50:04,905 - __main__ - DEBUG - Predicted: 9.12183952331543, True: 5.0

100%|██████████| 50/50 [1:00:28<00:00, 72.57s/trial, best loss: 44.98504367675202]
Best hyperparameters: {'colsample_bytree': np.float64(0.9059491926661292), 'early_stopping_rounds': np.float64(61.815051353004165), 'eta': np.float64(0.09521666051691245), 'gamma': np.float64(6.964586504082723), 'max_bin': np.float64(877.7357998919459), 'max_depth': np.float64(5.0), 'max_leaves': np.float64(213.76314799400666), 'min_child_weight': np.float64(3.0), 'n_estimators': np.float64(396.0), 'num_parallel_tree': np.float64(9.0), 'reg_alpha': np.float64(26.0), 'reg_lambda': np.float64(54.0), 'tree_method': np.int64(3)}


In [20]:
best: dict
best

{'colsample_bytree': np.float64(0.9059491926661292),
 'early_stopping_rounds': np.float64(61.815051353004165),
 'eta': np.float64(0.09521666051691245),
 'gamma': np.float64(6.964586504082723),
 'max_bin': np.float64(877.7357998919459),
 'max_depth': np.float64(5.0),
 'max_leaves': np.float64(213.76314799400666),
 'min_child_weight': np.float64(3.0),
 'n_estimators': np.float64(396.0),
 'num_parallel_tree': np.float64(9.0),
 'reg_alpha': np.float64(26.0),
 'reg_lambda': np.float64(54.0),
 'tree_method': np.int64(3)}

In [25]:
tree_method = ["auto", "exact", "approx", "hist"]
best["tree_method"] = tree_method[best["tree_method"]]
best["early_stopping_rounds"] = int(best["early_stopping_rounds"])
best["max_bin"] = int(best["max_bin"])
best["max_depth"] = int(best["max_depth"])
best["max_leaves"] = int(best["max_leaves"])
best["n_estimators"] = int(best["n_estimators"])
best["num_parallel_tree"] = int(best["num_parallel_tree"])

In [26]:
model = XGBRegressor(
    **best,
    eval_metric=mean_squared_error,
    n_jobs=-1,
    random_state=8888,
)

model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True,
)

predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)

print(f"Test MSE: {mse}")
print(f"Test RMSE: {np.sqrt(mse)}")
print(f"Test MAE: {np.mean(np.abs(y_test - predictions))}")

for pred, true in zip(predictions[:10], y_test[:10]):
    print(f"Predicted: {pred}, True: {true}")

[0]	validation_0-rmse:7.12157	validation_0-mean_squared_error:50.71675
[1]	validation_0-rmse:7.06056	validation_0-mean_squared_error:49.85153
[2]	validation_0-rmse:7.01901	validation_0-mean_squared_error:49.26656
[3]	validation_0-rmse:6.98399	validation_0-mean_squared_error:48.77605
[4]	validation_0-rmse:6.94492	validation_0-mean_squared_error:48.23186
[5]	validation_0-rmse:6.91467	validation_0-mean_squared_error:47.81273
[6]	validation_0-rmse:6.88544	validation_0-mean_squared_error:47.40924
[7]	validation_0-rmse:6.86999	validation_0-mean_squared_error:47.19680
[8]	validation_0-rmse:6.86202	validation_0-mean_squared_error:47.08736
[9]	validation_0-rmse:6.85347	validation_0-mean_squared_error:46.97011
[10]	validation_0-rmse:6.84419	validation_0-mean_squared_error:46.84288
[11]	validation_0-rmse:6.83640	validation_0-mean_squared_error:46.73637
[12]	validation_0-rmse:6.82127	validation_0-mean_squared_error:46.52977
[13]	validation_0-rmse:6.81364	validation_0-mean_squared_error:46.42569
[1